### Setup

In [1]:
import torch
import tifffile
from torchvision.transforms import ToTensor
from torch import nn
import numpy as np
from skimage.transform import resize
from skimage.exposure import rescale_intensity
from pathlib import Path

### Parameters

In [3]:
PATH = "model.pt"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

### Neural Network

In [4]:
from training_model import UNet

model = UNet(1024, n_channels=1, n_class=1).to(device)
print(f"Using {device} device")

Using cuda device


In [5]:
image = Path("../../data/1351/C39907/00/image.tif").resolve()

image = tifffile.imread(image).astype(np.float32)
image = resize(image, (128, 128, 128), order=0)
image = rescale_intensity(image, out_range=(0, 1))

image.shape

(128, 128, 128)

In [6]:
image_tensor = ToTensor()(image)
image_tensor = image_tensor[None]
image_tensor = image_tensor[None]
image_tensor = image_tensor.to(device)

print(image_tensor.shape)

torch.Size([1, 1, 128, 128, 128])


In [8]:
checkpoint = torch.load("../checkpoints1/all_data_100epochs_1024bottleneck_best.pt")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [28]:
out = model(image_tensor)
out = out.cpu().detach().numpy()
out = np.squeeze(out)
out.shape

(128, 128, 128)

In [29]:
import napari

viewer = napari.view_image(image)
viewer.add_image(np.transpose(out, axes=(1, 2, 0)), name="lungs prediction", visible=False)

<Image layer 'lungs prediction' at 0x7faecadd1ac0>

### Post processing

In [30]:
segmentation = out > 0.5

viewer.add_labels(np.transpose(segmentation, axes=(1, 2, 0)), name="segmentation", opacity=0.2)
im = viewer.add_labels(np.transpose(segmentation, axes=(1, 2, 0)), name="segmentation")
im.contour = 2

/home/qchappui/Documents/project/env/lib/python3.9/site-packages/napari/layers/labels/labels.py:966: UserWarning: Contours are not displayed during 3D rendering
  warnings.warn(
